In [2]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

In [3]:
user = 'admin'
password = 'isba4715'
host = 'isba-dev-01.cjwugys6yj0u.us-east-1.rds.amazonaws.com'
port_number = '3306'
database_name = 'HPolProject'
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port_number}/{database_name}')



In [11]:
#What is the total amount of fully dual eligible, partially, and not dually eligible? Make sure to make it the most recent (2022).
#We want to know whic
sql_query = """
SELECT 
	f.state,
    f.countenrolled AS Fully,
    p.countenrolled AS Partial,
    n.countenrolled AS Non
FROM 
    FullDualEligibility f
JOIN PartialDualEligibility p 
	ON f.state = p.state
JOIN NotDuallyEligible n 
	ON f.state = n.state 
WHERE 
    f.state IN (SELECT state FROM PartialDualEligibility)
    AND f.state IN (SELECT state FROM NotDuallyEligible)
    AND f.month > '2021-12-31'
GROUP BY 
    f.state;
"""

df = pd.read_sql_query(sql_query, engine)

df

#Insight: We have found that there is still substantial amount of people that are still in all categories
#Recommendation: Maintain or make policies in which could maintain this much people as there are adverse consequences to the complications of Medicare and Medicaid

,state,Fully,Partial,Non
0,Alabama,"92,228","139,986","920,630"
1,Alaska,"21,174","1,133","141,377"
2,Arizona,"231,172","56,286","1,847,624"
3,Arkansas,"87,432","66,162","1,022,638"
4,North Dakota,"14,307","2,550","59,028"
5,Oklahoma,"101,907","30,479","704,139"
6,New Jersey,"242,154",23,"1,520,614"
7,District of Columbia,"27,763","11,930","147,064"
8,Massachusetts,"352,863","34,296","1,514,411"
9,Colorado,"103,124","40,491","1,299,061"


In [13]:
#what then is the portion of people within the full eligiblity or partial elgibiility that are within identified major eligible groups.
    #Particularly what is it like in Washington
sql_query1 = """
WITH statematch AS (
SELECT 
    state,
    SUM(CASE WHEN majoreligibilitygroup = 'Adult' THEN counteverenrolled ELSE NULL END) AS Adult,
    SUM(CASE WHEN majoreligibilitygroup = 'Adult expansion group' THEN counteverenrolled ELSE NULL END) AS Adult_expansion_group,
    SUM(CASE WHEN majoreligibilitygroup = 'COVID newly-eligible' THEN counteverenrolled ELSE NULL END) AS COVID_Eligible,
    SUM(CASE WHEN majoreligibilitygroup = 'Children' THEN counteverenrolled ELSE NULL END) AS Children,
    SUM(CASE WHEN majoreligibilitygroup = 'Aged' THEN counteverenrolled ELSE NULL END) AS Aged,
    SUM(CASE WHEN majoreligibilitygroup = 'Persons with disabilities' THEN counteverenrolled ELSE NULL END) AS Disabled,
    SUM(CASE WHEN majoreligibilitygroup = 'Unknown' THEN counteverenrolled ELSE NULL END) AS Unknown
FROM MajorEligibilityGroupYr
WHERE `year` <2022
GROUP BY 
    state     
)
SELECT 
    s.state,
    s.Adult,
    s.Adult_expansion_group,
    s.COVID_Eligible,
    s.Children,
    s.Disabled,
    s.Unknown,
    p.countenrolled AS Partial,
    f.countenrolled AS Fully
FROM 
    statematch s
JOIN PartialDualEligibility p 
	ON s.state = p.state
JOIN FullDualEligibility f 
	ON s.state = f.state
GROUP BY 
	s.state;
"""


sql_query2 = """
WITH statematch AS (
SELECT 
    state,
    SUM(CASE WHEN majoreligibilitygroup = 'Adult' THEN counteverenrolled ELSE NULL END) AS Adult,
    SUM(CASE WHEN majoreligibilitygroup = 'Adult expansion group' THEN counteverenrolled ELSE NULL END) AS Adult_expansion_group,
    SUM(CASE WHEN majoreligibilitygroup = 'COVID newly-eligible' THEN counteverenrolled ELSE NULL END) AS COVID_Eligible,
    SUM(CASE WHEN majoreligibilitygroup = 'Children' THEN counteverenrolled ELSE NULL END) AS Children,
    SUM(CASE WHEN majoreligibilitygroup = 'Persons with disabilities' THEN counteverenrolled ELSE NULL END) AS Disabled,
    SUM(CASE WHEN majoreligibilitygroup = 'Unknown' THEN counteverenrolled ELSE NULL END) AS Unknown
FROM MajorEligibilityGroupYr
WHERE `year` <2022
GROUP BY 
    state     
)
SELECT 
    s.state,
    s.Adult,
    s.Adult_expansion_group,
    s.COVID_Eligible,
    s.Children,
    s.Disabled,
    s.Unknown,
    p.countenrolled AS Partial,
    f.countenrolled AS Fully
FROM 
    statematch s
JOIN PartialDualEligibility p 
	ON s.state = p.state
JOIN FullDualEligibility f 
	ON s.state = f.state
WHERE s.state = 'Washington'
GROUP BY 
	s.state;
"""


df = pd.read_sql_query(sql_query2, engine)

df


OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query ([Errno 110] Connection timed out)')
[SQL: 
WITH statematch AS (
SELECT 
    state,
    SUM(CASE WHEN majoreligibilitygroup = 'Adult' THEN counteverenrolled ELSE NULL END) AS Adult,
    SUM(CASE WHEN majoreligibilitygroup = 'Adult expansion group' THEN counteverenrolled ELSE NULL END) AS Adult_expansion_group,
    SUM(CASE WHEN majoreligibilitygroup = 'COVID newly-eligible' THEN counteverenrolled ELSE NULL END) AS COVID_Eligible,
    SUM(CASE WHEN majoreligibilitygroup = 'Children' THEN counteverenrolled ELSE NULL END) AS Children,
    SUM(CASE WHEN majoreligibilitygroup = 'Persons with disabilities' THEN counteverenrolled ELSE NULL END) AS Disabled,
    SUM(CASE WHEN majoreligibilitygroup = 'Unknown' THEN counteverenrolled ELSE NULL END) AS Unknown
FROM MajorEligibilityGroupYr
WHERE `year` <2022
GROUP BY 
    state     
)
SELECT 
    s.state,
    s.Adult,
    s.Adult_expansion_group,


In [ ]:
#Insight: It seems that washington does not have anyone under the partial or fully eligible category within the major eligibile groups
#Recommendation: To remain that way to reduce potential complications of eligibility